In [1]:
import os
from os import path
import sqlite3
global_sqlite_filename='apf.db'
import pandas
from datetime import datetime, timedelta
import numpy as np
from astropy.time import Time
from sqlalchemy import create_engine
import pytz
import numpy as np

In [2]:
class col_types:
    def __init__(self):
        self.TEXT = 'TEXT'
        self.INTEGER = 'INTEGER'
        self.REAL = 'REAL'
        self.BLOB = 'BLOB'
        self.NULL = 'NULL'
        
ct = col_types()
print(ct.TEXT)

#print(os.getcwd())
#print(os.listdir())

TEXT


In [3]:
def create_database(sqlite_filename=global_sqlite_filename):
    conn = sqlite3.connect(sqlite_filename)
    cursor = conn.cursor()
    return cursor
       
def get_telemetry_columns():
    columns = {'DateTime': ct.TEXT, 
                  'TARGET': ct.NULL,
                  'MIDPTFIN': ct.REAL,
                  'AZ': ct.REAL,
                  'EL': ct.REAL,
                  'AZENCPOS': ct.REAL,
                  'ELENCPOS': ct.REAL,
                  'AZENCVEL': ct.REAL,
                  'ELENCVEL': ct.REAL,
                  'AZFLWERR': ct.REAL,
                  'ELFLWERR': ct.REAL,
                  'OUTFILE': ct.REAL,
                  'OBSNUM': ct.REAL,
                  'MODE': ct.REAL,
                  'AVG_FWHM': ct.REAL,
                  'M5WIND': ct.REAL,
                  'M5WINDAZ': ct.REAL,
                  'TAVERAGE': ct.REAL,
                  'TM1S210': ct.REAL,
                  'TM2CAIR': ct.REAL,
                  'OFFSET_AZ': ct.REAL,
                  'OFFSET_EL': ct.REAL,
                  'RMSOFFSET_AZ': ct.REAL,
                  'RMSOFFSET_EL': ct.REAL,
                  'AVGOFFSET_AZ': ct.REAL,
                  'AVGOFFSET_EL': ct.REAL,
                  'HATCHPOS': ct.TEXT,
                  'EVENT': ct.TEXT}    
    return columns
    
def get_velocity_columns():
    columns = [('MdptJulian', ct.REAL),
               ('velocity', ct.REAL),
               ('velocity_error', ct.REAL),
               ('stellar_activity1', ct.REAL),
               ('stellar_activity2', ct.REAL),
               ('total_counts', ct.REAL),
               ('brad_doesnt_know', ct.REAL),
               ('DateTimeUTC', ct.TEXT),
               ('DateTimeLocal', ct.TEXT),
               ('starID', ct.TEXT)
              ]
    return columns
    
def create_telemetry_table(cursor):        
    columns = get_telemetry_columns()
    l = ['{} {}'.format(k,v) for (k, v) in columns.items()]
    print(len(l))
    full_list = ', '.join(l)
    sql_create_string = 'CREATE TABLE telemetry (' + full_list + ', PRIMARY KEY (DateTime))'
    print(sql_create_string)        
    cursor.execute(sql_create_string)

    return


def create_velocity_table(cursor):
    columns = get_velocity_columns()
    l = ['{} {}'.format(k,v) for (k, v) in columns]
    full_list = ', '.join(l)
    sql_create_string = 'CREATE TABLE velocity (' + full_list + ', PRIMARY KEY (DateTimeLocal))'
    print(sql_create_string)  
    cursor.execute(sql_create_string)

    return

def open_connection(sqlite_filename=global_sqlite_filename):
    dburi = 'file:{}?mode=rw'.format(sqlite_filename)
    conn = sqlite3.connect(dburi, uri=True)
    return conn
    

In [4]:
conn = open_connection()
cursor = conn.cursor()
#create_telemetry_table(cursor)
#create_velocity_table(cursor)

In [5]:
def populate_telemetry_table(conn, filenames):
    for csv_file in filenames:
        try:
            df = pandas.read_csv(csv_file, 
                               sep='\t', skiprows=1, header=None, names=get_telemetry_columns(), comment='#')
            df.to_sql('telemetry', conn, if_exists='append', index=False)
            
        except Exception as e:
            print(csv_file)
            print(e)

def populate_telemetry(conn=None, dirname=None, batchsize=5):
    filenames = [os.path.join(dirname, x) for x in os.listdir(dirname)]
    filenames.sort()
    
    def batch(iterable, n=1):
        l = len(iterable)
        for ndx in range(0, l, n):
            yield iterable[ndx : min(ndx + n, l)]
    
    for file_batch in batch(filenames, n=batchsize):
        populate_telemetry_table(conn, file_batch)
        
    


        

In [6]:
#populate_telemetry(conn, '../telemetry_data', batchsize=10)

In [7]:
def load_velocity_csv(dirname, filename):
    full_path = os.path.join(dirname, filename)
    columns = get_velocity_columns()[:-2] # leave off the last two column names
                                        # because we're calculating them below.
    df = pandas.read_csv(full_path, sep='\s+',
            names=[x[0] for x in columns]) 
                                                
    return df
    
    
california_tzinfo = pytz.timezone('US/Pacific')

# returns a string for a single julian date. Couldn't figure out how to vectorize.
def convert_julian_to_datetime_local(juliantime):
    times = Time(juliantime, format='jd')
    full_iso = times.to_datetime(timezone=california_tzinfo).isoformat()
    return full_iso[:-6]

# takes in an array of julian dates. Returns an array of local isot strings.
def make_array_of_local_datetimes(julian_array):
    output_array = output_array = np.chararray(len(julian_array), itemsize=26, unicode=True)
    for i, julian_time in enumerate(julian_array):
        output_array[i] = convert_julian_to_datetime_local(julian_time)
    return output_array
    

def convert_julian_to_datetime_utc(juliantimes):
    times = Time(juliantimes, format='jd')
    return times.isot 
    
def add_calculated_columns(df, filename):
    new_df = df.copy()
    #datetimes = convert_julian_to_datetime(new_df.MdptJulian)
    new_df['DateTimeUTC'] = convert_julian_to_datetime_utc(new_df.MdptJulian)
    new_df['DateTimeLocal'] = make_array_of_local_datetimes(new_df.MdptJulian)
    new_df['starID'] = [filename] * len(new_df.index) 
    
    return new_df
    

def populate_velocity(conn, dirname='../standard_star_velocity_measurements/', debug=False):
    filenames = ['HD10700_APF.vels', 'HD185144_APF.vels', 'HD9407_APF.vels']
    for file in filenames:
        print('started {}'.format(file))
        df_incomplete = load_velocity_csv(dirname, file)
        df_complete = add_calculated_columns(df_incomplete, file)
        if debug:
            print(df_complete)
        else: 
            df_complete.to_sql('velocity', conn, if_exists='append', index=False)
        print('tried to add {} entries to \'velocity\''.format(len(df_complete)))

#velocity_df = load_velocity_csv('../standard_star_velocity_measurements/', 'HD10700_APF.vels')
#add_calculated_columns(velocity_df, 'HD10700_APF.vels' )


In [9]:
#populate_velocity(conn)

started HD10700_APF.vels
tried to add 755 entries to 'velocity'
started HD185144_APF.vels
tried to add 1655 entries to 'velocity'
started HD9407_APF.vels
tried to add 694 entries to 'velocity'


In [8]:
# create additional calculated columns on velocity
engine = create_engine('sqlite:///apf.db', echo=False)


In [153]:
#add new table telemetrytimelookup
def drop_telemetry_time_lookup(engine):
    engine.execute("""DROP TABLE IF EXISTS telemetry_time_lookup;""")
    print('dropped telemetry_time_lookup.')

def create_telemetry_time_lookup(engine):
    engine.execute("""
    CREATE TABLE telemetry_time_lookup (
    DateTime TEXT PRIMARY KEY,
    YearMonthDayHour REAL,
    EpocSeconds REAL,
    Event TEXT,
    ExposureID TEXT,
    FOREIGN KEY (DateTime) REFERENCES telemetry(DateTime) 
    )""")
    print('created telemetry_time_lookup.')

def populate_telemetry_time_lookup(engine):   
    engine.execute("""INSERT INTO telemetry_time_lookup (DateTime,
        YearMonthDayHour, EpocSeconds, Event) 
    SELECT t.DateTime, strftime('%Y-%m-%d-%H', t.datetime), strftime('%s', t.DateTime), t.Event
    from telemetry t
    """)
    print('populated created telemetry_time_lookup.')
    

def create_index_in_telemetry_time_lookup(engine):
    engine.execute("""CREATE INDEX idx_yearmonthdayhour ON 
        telemetry_time_lookup(YearMonthDayHour asc, EpocSeconds asc)
    """)
    print('created index on YearMonthDayHour in TTC.')
    
#drop_telemetry_time_lookup(engine)
#create_telemetry_time_lookup(engine)
#populate_telemetry_time_lookup(engine)
#create_index_in_telemetry_time_lookup(engine)

df = pandas.read_sql_query("""select epocseconds - 1436589861.0, event from (select * from telemetry_time_lookup
    where YearMonthDayHour = '2015-07-11-04') 
    where event != 'ExposureBegin' 
    and epocseconds - 1436589861.0 < 0
    order by  abs(1436589861.0 - epocseconds) 
    limit 1
    """, con=engine)
pandas.set_option('precision', 18)
print(df)



created index on YearMonthDayHour in TTC.
   epocseconds - 1436589861.0       Event
0                       -38.0  EraseBegin


In [163]:
def drop_vel_telemetry_stats(engine):
    engine.execute("""DROP TABLE IF EXISTS vel_telemetry_stats;""")
    print('dropped vel_telemetry_stats.')

def create_vel_telemetry_stats(engine):
    engine.execute("""
    CREATE TABLE vel_telemetry_stats (
                  DateTimeLocal TEXT PRIMARY KEY,
                  YearMonthDayHour TEXT,
                  EpocSeconds REAL,
                  exposure_begin_datetime TEXT,
                  exposure_end_datetime TEXT,
                  Delta_temp_from_begin_night REAL, 
    FOREIGN KEY (DateTimeLocal) REFERENCES velocity(DateTimeLocal) 
    )""")
    print('created vel_telemetry_stats')
    return

def get_begin_end_exposure_datetimes(YearMonthDayHour, epocseconds, engine):
    df_begin = pandas.read_sql_query("""select epocseconds - {}, event from (select * from telemetry_time_lookup
    where YearMonthDayHour = {1}) 
    where event != 'ExposureBegin' 
    and epocseconds - {0} < 0
    order by  abs({0} - epocseconds) 
    limit 1
    """.format(), con=engine)
    
    df_end = pandas.read_sql_query("""select epocseconds - {}, event from (select * from telemetry_time_lookup
    where YearMonthDayHour = {1}) 
    where event != 'ExposureBegin' 
    and epocseconds - {0} > 0
    order by  abs({0} - epocseconds) 
    limit 1
    """.format(), con=engine)
    
    return df_begin, df_end

def populate_only_datetime_vts(engine):
    engine.execute("""Insert into vel_telemetry_stats
                                    (DateTimeLocal, YearMonthDayHour, EpocSeconds)
                                    select v.DateTimeLocal, 
                                        strftime('%Y-%m-%d-%H', v.DateTimeLocal), 
                                        strftime('%s', v.DateTimeLocal)
                                    from velocity v
                                    where v.DateTimeLocal > "2015-01-01T17:00:00" """)
    print(pandas.read_sql_query("""select * from vel_telemetry_stats order by datetimelocal desc limit 10""", engine))
    
def get_begin_end_times(engine, VelocityDateTime, printstuff=False):
    DateTime = VelocityDateTime
    if printstuff:
        print('starting datetime: {}'.format(DateTime))
        
    def make_sql(DateTime, TrueForBefore=True):
        if TrueForBefore:
            greaterthan = '<'
            ordering = 'desc'
        else:
            greaterthan = '>'
            ordering = 'asc'
        output = """select datetime, epocseconds - strftime("%s", "{0}"), event from 
            (select * from telemetry_time_lookup where 
            YearMonthDayHour =  strftime('%Y-%m-%d-%H', "{0}" ) 
        union all
            select * from telemetry_time_lookup where 
            YearMonthDayHour = (select yearmonthdayhour from telemetry_time_lookup 
                where YearMonthDayHour {1} strftime('%Y-%m-%d-%H', "{0}") order by 
                YearMonthDayHour {2} limit 1)
            ) 
         where event != 'ExposureBegin' 
            and epocseconds - strftime("%s", "{0}") {1} 0
            order by  abs(strftime("%s", "{0}") - epocseconds) 
            limit 1""".format(DateTime, greaterthan, ordering)
        
        return output
    
    if printstuff:
        print(make_sql(VelocityDateTime, TrueForBefore=True))
        print(make_sql(VelocityDateTime, TrueForBefore=False))
    
    df_begin = pandas.read_sql_query(make_sql(VelocityDateTime, TrueForBefore=True), engine)
    
    df_end = pandas.read_sql_query(make_sql(VelocityDateTime, TrueForBefore=False), engine)
  
    no_telemetry_message = 'no telemetry for this time'
    if df_begin.empty:
        first_timestamp = no_telemetry_message
        #raise Exception('df_begin empty for datetimelocal= {}'.format(VelocityDateTime))
    else:
        just_prior_timestamp = df_begin.DateTime[0]
        first_timestamp =  pandas.read_sql_query("""
            select DateTime from telemetry_time_lookup where
            datetime > "{}" limit 1
            """.format(just_prior_timestamp), engine).DateTime[0]
        
    if df_end.empty:
        last_timestamp = no_telemetry_message
        #raise Exception('df_end empty for datetimelocal= {}'.format(VelocityDateTime))
    else:
        just_after_timestamp = df_end.DateTime[0]
        last_timestamp =  pandas.read_sql_query("""
            select DateTime from telemetry_time_lookup where
            datetime < "{}" order by DateTime desc limit 1
            """.format(just_after_timestamp), engine).DateTime[0]
        

    if printstuff:
        print(just_prior_timestamp, just_after_timestamp)
        print(first_timestamp, last_timestamp)
    
    return first_timestamp, last_timestamp
    

def populate_begin_end_times(engine, printstuff=False):
    
    
    velocity_times = pandas.read_sql_query("""
               select v.datetimelocal from velocity v
                 inner join vel_telemetry_stats vt
                 on v.datetimelocal = vt.datetimelocal
                 where v.datetimelocal > "2015-01-01T00:00:58.463989"
                 and vt.exposure_begin_datetime is null
    """, engine).DateTimeLocal 
    
    for i, v_timestamp in enumerate(velocity_times):
        if printstuff:
            print('trying {}/{}: {}'.format(i, len(velocity_times), v_timestamp))
        begin_time, end_time = get_begin_end_times(engine, v_timestamp)
        if printstuff:
            print('{} {}'.format(begin_time, end_time))
        engine.execute("""UPDATE vel_telemetry_stats 
                            SET exposure_begin_datetime = "{exposure_begin}",
                                exposure_end_datetime = "{exposure_end}"
                            WHERE DateTimeLocal = "{v_timestamp}"
                            """.format(exposure_begin=begin_time, 
                                      exposure_end=end_time,
                                      v_timestamp=v_timestamp))
    
    print("just tried to populate {} rows of begin-end timestamps".format(len(velocity_times)))

def populate_vel_telemetry_stats_from_telemetry(engine):
    

#drop_vel_telemetry_stats(engine)
#create_vel_telemetry_stats(engine)
#populate_only_datetime_vts(engine)
#populate_begin_end_times(engine, printstuff=True) #this takes a while.


In [191]:
def drop_final(engine):
    engine.execute("""DROP TABLE IF EXISTS final;""")
    print('dropped final.')

def create_final(engine):
    engine.execute("""
    CREATE TABLE final (
                  DateTimeLocal TEXT PRIMARY KEY,
                  YearMonthDayHour TEXT,
                  EpocSeconds REAL,
                  exposure_begin_datetime TEXT,
                  exposure_end_datetime TEXT,
                  velocity REAL,
                  velocity_error REAL,
                  stellar_activity1 REAL,
                  stellar_activity2 REAL,
                  total_counts INTEGER,
                  brad_doesnt_know INTEGER,
                  DateTimeUTC TEXT,
                  starID TEXT,
                  MIDPTFIN REAL,
                  AZ REAL,
                  EL REAL,
                  AZENCPOS REAL,
                  ELENCPOS REAL,
                  AZENCVEL REAL,
                  ELENCVEL REAL,
                  AZFLWERR REAL,
                  ELFLWERR REAL,
                  OUTFILE REAL,
                  OBSNUM REAL,
                  MODE REAL,
                  AVG_FWHM REAL,
                  M5WIND REAL,
                  M5WINDAZ REAL,
                  TAVERAGE REAL,
                  TM1S210 REAL,
                  TM2CAIR REAL,
                  OFFSET_AZ REAL,
                  OFFSET_EL REAL,
                  RMSOFFSET_AZ REAL,
                  RMSOFFSET_EL REAL,
                  AVGOFFSET_AZ REAL,
                  AVGOFFSET_EL REAL,
                  HATCHPOS TEXT,
                  Delta_temp_from_begin_night REAL, 
    FOREIGN KEY (DateTimeLocal) REFERENCES velocity(DateTimeLocal) 
    )""")
    print('created final')
    return


    
def populate_final(engine):
    engine.execute("""
        insert into final 
            (     DateTimeLocal,
                  YearMonthDayHour,
                  EpocSeconds,
                  exposure_begin_datetime,
                  exposure_end_datetime,
                  velocity,
                  velocity_error,
                  stellar_activity1,
                  stellar_activity2,
                  total_counts,
                  brad_doesnt_know,
                  DateTimeUTC,
                  starID,
                  MIDPTFIN,
                  AZ,
                  EL,
                  AZENCPOS,
                  ELENCPOS,
                  AZENCVEL,
                  ELENCVEL,
                  AZFLWERR,
                  ELFLWERR,
                  OUTFILE,
                  OBSNUM,
                  MODE,
                  AVG_FWHM,
                  M5WIND,
                  M5WINDAZ,
                  TAVERAGE,
                  TM1S210,
                  TM2CAIR,
                  OFFSET_AZ,
                  OFFSET_EL,
                  RMSOFFSET_AZ,
                  RMSOFFSET_EL,
                  AVGOFFSET_AZ,
                  AVGOFFSET_EL,
                  HATCHPOS)
        select 
                  vts.DateTimeLocal,
                  vts.YearMonthDayHour,
                  vts.EpocSeconds,
                  vts.exposure_begin_datetime,
                  vts.exposure_end_datetime,
                  v.velocity,
                  v.velocity_error,
                  v.stellar_activity1,
                  v.stellar_activity2,
                  v.total_counts,
                  v.brad_doesnt_know,
                  v.DateTimeUTC,
                  v.starID,
                  t.MIDPTFIN,
                  t.AZ,
                  t.EL,
                  t.AZENCPOS,
                  t.ELENCPOS,
                  t.AZENCVEL,
                  t.ELENCVEL,
                  t.AZFLWERR,
                  t.ELFLWERR,
                  t.OUTFILE,
                  t.OBSNUM,
                  t.MODE,
                  t.AVG_FWHM,
                  t.M5WIND,
                  t.M5WINDAZ,
                  t.TAVERAGE,
                  t.TM1S210,
                  t.TM2CAIR,
                  t.OFFSET_AZ,
                  t.OFFSET_EL,
                  t.RMSOFFSET_AZ,
                  t.RMSOFFSET_EL,
                  t.AVGOFFSET_AZ,
                  t.AVGOFFSET_EL,
                  t.HATCHPOS
            from 
                vel_telemetry_stats vts
               INNER JOIN 
               telemetry t
               ON vts.exposure_end_datetime = t.DateTime
               INNER JOIN 
               velocity v
               ON vts.DateTimeLocal = v.DateTimeLocal
            
                  """)
    print('populated final.')

def test_final(engine):
    df1 = pandas.read_sql_query("""
        select * from final
        limit 10""", engine)
    df2 = pandas.read_sql_query("""
        select count(*) from final
        limit 10""", engine)
    df3 = pandas.read_sql_query("""
        select exposure_end_datetime from final
         order by exposure_end_datetime""", engine)
    print(df, df2, df3)    

#drop_final(engine)
#create_final(engine)
#populate_final(engine)
#test_final(engine)